In [13]:
%pip install pennylane scikit-learn numpy matplotlib
%pip install qiskit
%pip install qiskit[visualization] 
%pip install qiskit-aer
%pip install qiskit-ibm-runtime
%pip install qiskit-algorithms
%pip install qiskit-machine-learning
%pip install pylatexenc

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ------------ --------------------------- 2.1/6.5 MB 11.2 MB/s eta 0:00:01
   ------------------------- -------------- 4.2/6.5 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 10.6 MB/s  0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   -- ------------------------------------- 2.1/41.0 MB 11.8 MB/s eta 0:00:04
   ---- ----------------------------------- 4.5/41.0 MB 11.7 MB/s eta 0:00:04
   ------ ---

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

DATASETS = {
    "concrete_lof": {
        "path": "concrete_cleanLOF.csv",
        "features": ["Cement","BlastFurnaceSlag","FlyAsh","Water","Superplasticizer","CoarseAggregate","FineAggregate","Age"],
        "target": "ConcreteCompressiveStrength",
        "outlier_flag": "is_outlier",
        "rename": None,          
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "baseball": {
        "path": "baseball_cleanLOF.csv",
        "features": ["Batting_average","On-base_percentage","Runs","Hits","Doubles","Triples","HomeRuns","Runs_batted_in","Walks","Strike-Outs","Stolen_bases","Errors","Free_agency_eligibility","Free_agent","Arbitration_eligibility","Arbitration","Salary"],
        "target": None,          
        "outlier_flag": "is_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "concrete_ocsvm": {
        "path": "concrete_cleanOCSVM.csv",
        "features": ["Cement","BlastFurnaceSlag","FlyAsh","Water","Superplasticizer","CoarseAggregate","FineAggregate","Age"],
        "target": "ConcreteCompressiveStrength",
        "outlier_flag": "ocsvm_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "elevators": {
        "path": "Elevators_cleanLOF.csv",
        "features": ["ClimbRate","Sgz","P","Q","CurRoll","AbsRoll","DiffClb","DiffRollRate","DiffDiffClb","SaTime1","SaTime2","SaTime3","SaTime4","DiffSaTime1","DiffSaTime2","DiffSaTime3","DiffSaTime4","Sa","Goal"],
        "target": "Goal",
        "outlier_flag": "is_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
}

def load_dataset(spec_name: str):
    spec = DATASETS[spec_name]
    path = Path(spec["path"])
    if not path.exists():
        raise FileNotFoundError(f"No se encontró {path.resolve()}")
    df = pd.read_csv(path)
    if spec.get("rename"):
        df = df.rename(columns=spec["rename"])
    feats = spec["features"]
    missing = [c for c in feats if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en el CSV: {missing}")
    X = df[feats].copy()
    imputer = SimpleImputer(strategy="median")
    X = pd.DataFrame(imputer.fit_transform(X), columns=feats)

    if spec["scaling"].get("standard"):
        X = pd.DataFrame(StandardScaler().fit_transform(X), columns=feats)
    if spec["scaling"].get("minmax"):
        X = pd.DataFrame(MinMaxScaler(feature_range=spec["scaling"]["minmax"]).fit_transform(X), columns=feats)

    X = X.to_numpy(dtype=np.float64)

    tgt = spec.get("target")
    flag = spec.get("outlier_flag")

    meta = {
        "name": spec_name,
        "path": str(path),
        "features": feats,
        "target": tgt,
        "outlier_flag": flag,
        "N": X.shape[0],
        "d": X.shape[1],
    }
    return X, meta

In [3]:
DATASET_NAME = "concrete_lof"  # "concrete_lof" | "baseball" | "concrete_ocsvm" | "elevators"
X, meta = load_dataset(DATASET_NAME)
print(meta)

{'name': 'concrete_lof', 'path': 'concrete_cleanLOF.csv', 'features': ['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'Superplasticizer', 'CoarseAggregate', 'FineAggregate', 'Age'], 'target': 'ConcreteCompressiveStrength', 'outlier_flag': 'is_outlier', 'N': 978, 'd': 8}


In [4]:
import numpy as np
import pennylane as qml

assert 'X' in globals()
N, d = X.shape

def binarize_sign(x: np.ndarray) -> np.ndarray:
    return (x > 0).astype(int)

def _normalize_pad_pow2(v):
    v = np.array(v, dtype=float).ravel()
    n = int(np.ceil(np.log2(max(1, len(v)))))
    size = 2**n
    if len(v) < size:
        v = np.pad(v, (0, size - len(v)))
    norm = np.linalg.norm(v)
    if norm == 0:
        v = np.zeros_like(v)
        v[0] = 1.0
    else:
        v = v / norm
    return v

def angle_embed_manual(x, wires, rotation='Y'):
    rot = rotation.upper()
    gate = {'X': qml.RX, 'Y': qml.RY, 'Z': qml.RZ}.get(rot, None)
    for wi, xi in zip(wires, x):
        gate(float(xi), wires=wi)

def basis_embed_manual(bits, wires):
    for wi, bi in zip(wires, bits):
        if int(bi) == 1:
            qml.PauliX(wires=wi)

def amplitude_embed_manual_from_features(x, wires):
    v = _normalize_pad_pow2(x)
    if len(v) != 2**len(wires):
        raise ValueError("Número de wires insuficiente para vector de amplitudes.")
    qml.StatePrep(v, wires)

qubits_angle = d
wires_angle = list(range(qubits_angle))
dev_angle = qml.device("default.qubit", wires=wires_angle, shots=None)

qubits_basis = d
wires_basis = list(range(qubits_basis))
dev_basis = qml.device("default.qubit", wires=wires_basis, shots=None)

qubits_amp = int(np.ceil(np.log2(d)))
wires_amp = list(range(qubits_amp))
dev_amp = qml.device("default.qubit", wires=wires_amp, shots=None)

@qml.qnode(dev_angle, diff_method=None)
def manual_angle_qnode(x):
    angle_embed_manual(x, wires=wires_angle, rotation='Y')
    return qml.state()

@qml.qnode(dev_basis, diff_method=None)
def manual_basis_qnode(x):
    bits = binarize_sign(x)[:len(wires_basis)]
    basis_embed_manual(bits, wires=wires_basis)
    return qml.probs(wires=wires_basis)

@qml.qnode(dev_amp, diff_method=None)
def manual_amplitude_qnode(x):
    amplitude_embed_manual_from_features(x, wires=wires_amp)
    return qml.state()

In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

assert 'X' in globals()
N, d = X.shape

def _normalize_pad_pow2(v):
    v = np.array(v, dtype=float).ravel()
    n = int(np.ceil(np.log2(max(1, len(v)))))
    size = 2**n
    if len(v) < size:
        v = np.pad(v, (0, size - len(v)))
    norm = np.linalg.norm(v)
    if norm == 0:
        v = np.zeros_like(v); v[0] = 1.0
    else:
        v = v / norm
    return v

def qc_angle(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.ry(float(xi), i)
    return qc

def qc_basis(x):
    qc = QuantumCircuit(d)
    bits = (x > 0).astype(int)
    for i, bi in enumerate(bits):
        if int(bi) == 1:
            qc.x(i)
    return qc

def qc_amplitude(x):
    n = int(np.ceil(np.log2(d)))
    qc = QuantumCircuit(n)
    v = _normalize_pad_pow2(x)
    psi = v.astype(complex)
    qc.initialize(psi, list(range(n)))
    return qc

def sv_angle(x):
    return Statevector.from_instruction(qc_angle(x))

def sv_basis(x):
    return Statevector.from_instruction(qc_basis(x))

def sv_amplitude(x):
    return Statevector.from_instruction(qc_amplitude(x))

In [7]:
import pennylane as qml

assert 'X' in globals()
N, d = X.shape

wires = list(range(d))
dev = qml.device("default.qubit", wires=wires, shots=None)

def fourier_map(x, wires):
    for i, xi in enumerate(x[:len(wires)]):
        qml.RZ(float(xi), wires=wires[i])
    qml.QFT(wires=wires)

def zz_feature_map(x, wires):
    m = len(wires)
    for i, xi in enumerate(x[:m]):
        qml.RZ(float(xi), wires=wires[i])
    for i in range(m):
        for j in range(i+1, m):
            qml.IsingZZ(2.0*float(x[i%len(x)])*float(x[j%len(x)]), wires=[wires[i], wires[j]])

def pauli_feature_map(x, wires):
    m = len(wires)
    for i, xi in enumerate(x[:m]):
        qml.RX(float(xi), wires=wires[i])
        qml.RZ(float(xi), wires=wires[i])
    for i in range(m):
        for j in range(i+1, m):
            qml.IsingXX(2.0*float(x[i%len(x)])*float(x[j%len(x)]), wires=[wires[i], wires[j]])
            qml.IsingZZ(2.0*float(x[i%len(x)])*float(x[j%len(x)]), wires=[wires[i], wires[j]])

@qml.qnode(dev, diff_method=None)
def fm_fourier_state(x):
    fourier_map(x)
    return qml.state()

@qml.qnode(dev, diff_method=None)
def fm_zz_state(x):
    zz_feature_map(x)
    return qml.state()

@qml.qnode(dev, diff_method=None)
def fm_pauli_state(x):
    pauli_feature_map(x)
    return qml.state()

In [8]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT, RZZGate, RXXGate

assert 'X' in globals()
N, d = X.shape

def qc_fourier_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rz(float(xi), i)
    qc.compose(QFT(d, do_swaps=True), inplace=True)
    return qc

def qc_zz_feature_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rz(float(xi), i)
    for i in range(d):
        for j in range(i+1, d):
            qc.append(RZZGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
    return qc

def qc_pauli_feature_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rx(float(xi), i)
        qc.rz(float(xi), i)
    for i in range(d):
        for j in range(i+1, d):
            qc.append(RXXGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
            qc.append(RZZGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
    return qc

def sv_fourier(x):
    return Statevector.from_instruction(qc_fourier_map(x))

def sv_zz(x):
    return Statevector.from_instruction(qc_zz_feature_map(x))

def sv_pauli(x):
    return Statevector.from_instruction(qc_pauli_feature_map(x))

In [ ]:
import pennylane as qml

def ring_pairs(n):
    return [(i,(i+1)%n) for i in range(n)]

def vqr_ansatz_pl(params, wires, layers):
    k=0
    for _ in range(layers):
        for i in wires:
            qml.RY(params[k],wires=i); k+=1
            qml.RZ(params[k],wires=i); k+=1
        for a,b in ring_pairs(len(wires)):
            qml.CZ(wires=[wires[a],wires[b]])

In [11]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def ring_pairs(n):
    return [(i,(i+1)%n) for i in range(n)]

def vqr_ansatz_qk(n_qubits, layers, prefix="theta"):
    params=ParameterVector(prefix, layers*2*n_qubits)
    qc=QuantumCircuit(n_qubits)
    k=0
    for _ in range(layers):
        for i in range(n_qubits):
            qc.ry(params[k],i); k+=1
            qc.rz(params[k],i); k+=1
        for a,b in ring_pairs(n_qubits):
            qc.cz(a,b)
    return qc, params

# VQC + Angle Embedding (Pennylane)

In [ ]:
import numpy as np, pandas as pd, pennylane as qml
from pennylane import numpy as pnp
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'angle_embed_manual' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
def to_unit(t): return (2.0*(t - ymin)/(ymax - ymin)) - 1.0
def from_unit(z): return ymin + (z + 1.0)*(ymax - ymin)/2.0
yu = to_unit(y)

wires = list(range(d))
dev = qml.device("default.qubit", wires=wires)
L = 2

@qml.qnode(dev, diff_method="best")
def model_u(x, params):
    angle_embed_manual(x, wires, rotation='Y')
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = pnp.array(np.random.default_rng(7).normal(0,0.1,size=L*2*d), requires_grad=True)
opt = qml.AdamOptimizer(0.02)
epochs = 60
bs = min(32, N)
idx = np.arange(N)

for _ in range(epochs):
    np.random.shuffle(idx)
    for i in range(0, N, bs):
        mb = idx[i:i+bs]
        def loss(p):
            preds = pnp.array([model_u(x, p) for x in X[mb]])
            return pnp.mean((preds - yu[mb])**2)
        theta = opt.step(loss, theta)

preds_u = np.array([model_u(x, theta) for x in X])
yhat = from_unit(preds_u)
print("MSE_PL_angle:", float(np.mean((yhat - y)**2)))
print(qml.draw(model_u)(X[0], theta))

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\pennylane\_grad.py:310: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


{'MSE_train': 275.9366989726107, 'MSE_test': 255.95483237531792, 'a': 1.0, 'b': 35.824961684143226}
0: ─╭AngleEmbedding(M0)──RY(0.00)───RZ(0.03)──╭●───────────────────╭Z──RY(-0.13)──RZ(-0.05)─╭● ···
1: ─├AngleEmbedding(M0)──RY(-0.03)──RZ(-0.09)─╰Z─╭●────────────────│───RY(-0.19)──RZ(-0.13)─╰Z ···
2: ─├AngleEmbedding(M0)──RY(-0.05)──RZ(-0.10)────╰Z─╭●─────────────│───RY(-0.18)──RZ(-0.02)─── ···
3: ─├AngleEmbedding(M0)──RY(0.01)───RZ(0.13)────────╰Z─╭●──────────│───RY(-0.13)──RZ(0.03)──── ···
4: ─├AngleEmbedding(M0)──RY(-0.05)──RZ(-0.06)──────────╰Z─╭●───────│───RY(0.02)───RZ(-0.02)─── ···
5: ─├AngleEmbedding(M0)──RY(0.05)───RZ(0.04)──────────────╰Z─╭●────│───RY(-0.25)──RZ(-0.05)─── ···
6: ─├AngleEmbedding(M0)──RY(0.01)───RZ(-0.09)────────────────╰Z─╭●─│───RY(-0.00)──RZ(0.01)──── ···
7: ─╰AngleEmbedding(M0)──RY(-0.00)──RZ(0.07)────────────────────╰Z─╰●──RY(-0.15)──RZ(-0.05)─── ···

0: ··· ───────────────────╭Z─┤  <Z>
1: ··· ─╭●────────────────│──┤  <Z>
2: ··· ─╰Z─╭●─────────────│──┤  <Z>

# VQC + Basis Embedding (Pennylane)

In [46]:
import numpy as np, pandas as pd, pennylane as qml
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'basis_embed_manual' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
wires = list(range(d))
dev = qml.device("default.qubit", wires=wires, shots=None)
L = 2

@qml.qnode(dev, diff_method="best")
def model(x, params):
    bits = (x[:len(wires)]>0).astype(int)
    basis_embed_manual(bits, wires)
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
opt = qml.AdamOptimizer(0.04)
for _ in range(40):
    theta = opt.step(lambda p: np.mean(( (ymin+(np.array([model(x,p) for x in X])+1)*(ymax-ymin)/2.0) - y )**2), theta)

yhat = ymin+(np.array([model(x,theta) for x in X])+1)*(ymax-ymin)/2.0
print("MSE_PL_basis:", float(np.mean((yhat-y)**2)))
print(qml.draw(model)(X[0], theta))

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\pennylane\_grad.py:310: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


KeyboardInterrupt: 

# VQC + Amplitude Embedding (Pennylane)

In [12]:
import numpy as np, pandas as pd, pennylane as qml
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'amplitude_embed_manual_from_features' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
n = int(np.ceil(np.log2(d)))
wires = list(range(n))
dev = qml.device("default.qubit", wires=wires, shots=None)
L = 3

@qml.qnode(dev, diff_method="best")
def model(x, params):
    amplitude_embed_manual_from_features(x, wires)
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*n)
opt = qml.AdamOptimizer(0.04)
for _ in range(40):
    theta = opt.step(lambda p: np.mean(( (ymin+(np.array([model(x,p) for x in X])+1)*(ymax-ymin)/2.0) - y )**2), theta)

yhat = ymin+(np.array([model(x,theta) for x in X])+1)*(ymax-ymin)/2.0
print("MSE_PL_amplitude:", float(np.mean((yhat-y)**2)))
print(qml.draw(model)(X[0], theta))

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\pennylane\_grad.py:310: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


MSE_PL_amplitude: 313.5084709321279
0: ─╭|Ψ⟩──RY(0.00)───RZ(0.03)──╭●────╭Z──RY(0.01)───RZ(0.13)──╭●────╭Z──RY(0.01)───RZ(-0.09)─╭● ···
1: ─├|Ψ⟩──RY(-0.03)──RZ(-0.09)─╰Z─╭●─│───RY(-0.05)──RZ(-0.06)─╰Z─╭●─│───RY(-0.00)──RZ(0.07)──╰Z ···
2: ─╰|Ψ⟩──RY(-0.05)──RZ(-0.10)────╰Z─╰●──RY(0.05)───RZ(0.04)─────╰Z─╰●──RY(-0.13)──RZ(-0.05)─── ···

0: ··· ────╭Z─┤  <Z>
1: ··· ─╭●─│──┤     
2: ··· ─╰Z─╰●─┤     


# VQC + Angle Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_angle' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
L = 2

def expval_z0(x, params):
    qc = qc_angle(x)
    qc_ans, pv = vqr_ansatz_qk(d, L)
    bound = qc_ans.bind_parameters({pv[i]: params[i] for i in range(len(pv))})
    qc = qc.compose(bound)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(d-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
def loss(p): 
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':120, 'xatol':1e-3, 'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_angle:", float(np.mean((yhat-y)**2)))
print((qc_angle(X[0]).compose(vqr_ansatz_qk(d,L)[0].bind_parameters({vqr_ansatz_qk(d,L)[1][i]:theta[i] for i in range(L*2*d)}))).draw("text"))

# VQC + Basis Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_basis' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
L = 2

def expval_z0(x, params):
    qc = qc_basis(x)
    qc_ans, pv = vqr_ansatz_qk(d, L)
    bind = qc_ans.bind_parameters({pv[i]: float(params[i]) for i in range(len(pv))})
    qc = qc.compose(bind)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(d-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
def loss(p):
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':120,'xatol':1e-3,'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_basis:", float(np.mean((yhat-y)**2)))
print((qc_basis(X[0]).compose(vqr_ansatz_qk(d,L)[0].bind_parameters({vqr_ansatz_qk(d,L)[1][i]:float(theta[i]) for i in range(L*2*d)}))).draw("text"))


# VQC + Amplitude Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_amplitude' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
n = int(np.ceil(np.log2(d)))
L = 2

def expval_z0(x, params):
    qc = qc_amplitude(x)
    qc_ans, pv = vqr_ansatz_qk(n, L)
    bind = qc_ans.bind_parameters({pv[i]: float(params[i]) for i in range(len(pv))})
    qc = qc.compose(bind)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(n-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*n)
def loss(p):
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':160,'xatol':1e-3,'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_amplitude:", float(np.mean((yhat-y)**2)))
print((qc_amplitude(X[0]).compose(vqr_ansatz_qk(n,L)[0].bind_parameters({vqr_ansatz_qk(n,L)[1][i]:float(theta[i]) for i in range(L*2*n)}))).draw("text"))

# VQC + Fourier Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd, pennylane as qml
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'fourier_map' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
wires = list(range(d))
dev = qml.device("default.qubit", wires=wires, shots=None)
L = 2

@qml.qnode(dev, diff_method="best")
def model(x, params):
    fourier_map(x, wires)
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
opt = qml.AdamOptimizer(0.04)
for _ in range(40):
    theta = opt.step(lambda p: np.mean(((ymin+(np.array([model(x,p) for x in X])+1)*(ymax-ymin)/2.0)-y)**2), theta)

yhat = ymin+(np.array([model(x,theta) for x in X])+1)*(ymax-ymin)/2.0
print("MSE_PL_fourier:", float(np.mean((yhat-y)**2)))
print(qml.draw(model)(X[0], theta))

# VQC + ZZ Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd, pennylane as qml
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'zz_feature_map' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
wires = list(range(d))
dev = qml.device("default.qubit", wires=wires, shots=None)
L = 2

@qml.qnode(dev, diff_method="best")
def model(x, params):
    zz_feature_map(x, wires)
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
opt = qml.AdamOptimizer(0.04)
for _ in range(40):
    theta = opt.step(lambda p: np.mean(((ymin+(np.array([model(x,p) for x in X])+1)*(ymax-ymin)/2.0)-y)**2), theta)

yhat = ymin+(np.array([model(x,theta) for x in X])+1)*(ymax-ymin)/2.0
print("MSE_PL_zz:", float(np.mean((yhat-y)**2)))
print(qml.draw(model)(X[0], theta))

# VQC + Pauli Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd, pennylane as qml
from pathlib import Path

assert 'X' in globals() and 'meta' in globals() and 'pauli_feature_map' in globals() and 'vqr_ansatz_pl' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
wires = list(range(d))
dev = qml.device("default.qubit", wires=wires, shots=None)
L = 2

@qml.qnode(dev, diff_method="best")
def model(x, params):
    pauli_feature_map(x, wires)
    vqr_ansatz_pl(params, wires, L)
    return qml.expval(qml.PauliZ(wires[0]))

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
opt = qml.AdamOptimizer(0.04)
for _ in range(40):
    theta = opt.step(lambda p: np.mean(((ymin+(np.array([model(x,p) for x in X])+1)*(ymax-ymin)/2.0)-y)**2), theta)

yhat = ymin+(np.array([model(x,theta) for x in X])+1)*(ymax-ymin)/2.0
print("MSE_PL_pauli:", float(np.mean((yhat-y)**2)))
print(qml.draw(model)(X[0], theta))

# VQC + Fourier Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_fourier_map' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
L = 2

def expval_z0(x, params):
    qc = qc_fourier_map(x)
    qc_ans, pv = vqr_ansatz_qk(d, L)
    bind = qc_ans.bind_parameters({pv[i]: float(params[i]) for i in range(len(pv))})
    qc = qc.compose(bind)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(d-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
def loss(p):
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':140,'xatol':1e-3,'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_fourier:", float(np.mean((yhat-y)**2)))
print((qc_fourier_map(X[0]).compose(vqr_ansatz_qk(d,L)[0].bind_parameters({vqr_ansatz_qk(d,L)[1][i]:float(theta[i]) for i in range(L*2*d)}))).draw("text"))

# VQC + ZZ Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_zz_feature_map' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
L = 2

def expval_z0(x, params):
    qc = qc_zz_feature_map(x)
    qc_ans, pv = vqr_ansatz_qk(d, L)
    bind = qc_ans.bind_parameters({pv[i]: float(params[i]) for i in range(len(pv))})
    qc = qc.compose(bind)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(d-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
def loss(p):
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':160,'xatol':1e-3,'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_zz:", float(np.mean((yhat-y)**2)))
print((qc_zz_feature_map(X[0]).compose(vqr_ansatz_qk(d,L)[0].bind_parameters({vqr_ansatz_qk(d,L)[1][i]:float(theta[i]) for i in range(L*2*d)}))).draw("text"))

# VQC + Pauli Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from qiskit.quantum_info import Statevector

assert 'X' in globals() and 'meta' in globals() and 'qc_pauli_feature_map' in globals() and 'vqr_ansatz_qk' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)
ymin, ymax = float(y.min()), float(y.max())
L = 2

def expval_z0(x, params):
    qc = qc_pauli_feature_map(x)
    qc_ans, pv = vqr_ansatz_qk(d, L)
    bind = qc_ans.bind_parameters({pv[i]: float(params[i]) for i in range(len(pv))})
    qc = qc.compose(bind)
    sv = Statevector.from_instruction(qc).data
    probs = np.abs(sv)**2
    s = 0.0
    for i,p in enumerate(probs):
        b = (i>>(d-1)) & 1
        s += (1 if b==0 else -1)*p
    return float(s)

theta = np.random.default_rng(7).normal(0,0.1,size=L*2*d)
def loss(p):
    preds = np.array([expval_z0(x,p) for x in X])
    preds = ymin+(preds+1)*(ymax-ymin)/2.0
    return float(np.mean((preds-y)**2))

from scipy.optimize import minimize
res = minimize(loss, theta, method='Nelder-Mead', options={'maxiter':160,'xatol':1e-3,'fatol':1e-3})
theta = res.x
preds = np.array([expval_z0(x,theta) for x in X])
yhat = ymin+(preds+1)*(ymax-ymin)/2.0
print("MSE_QK_pauli:", float(np.mean((yhat-y)**2)))
print((qc_pauli_feature_map(X[0]).compose(vqr_ansatz_qk(d,L)[0].bind_parameters({vqr_ansatz_qk(d,L)[1][i]:float(theta[i]) for i in range(L*2*d)}))).draw("text"))

# SVM + Angle Embedding (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'manual_angle_qnode' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [manual_angle_qnode(x) for x in X]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_angle:", float(np.mean((yhat-y)**2)))

# SVM + Basis Embedding (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'manual_basis_qnode' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

probs = [manual_basis_qnode(x) for x in X]
phis = [np.sqrt(p).astype(complex) for p in probs]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_basis:", float(np.mean((yhat-y)**2)))

# SVM + Amplitude Embedding (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'manual_amplitude_qnode' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [manual_amplitude_qnode(x) for x in X]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_amplitude:", float(np.mean((yhat-y)**2)))

# SVM + Fourier Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'fm_fourier_state' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [fm_fourier_state(x) for x in X]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_fourier:", float(np.mean((yhat-y)**2)))

# SVM + ZZ Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'fm_zz_state' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [fm_zz_state(x) for x in X]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_zz:", float(np.mean((yhat-y)**2)))

# SVM + Pauli Feature Map (PennyLane)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'fm_pauli_state' in globals()
N, d = X.shape
path = Path(meta["path"])
y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64) if meta.get("target") else (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [fm_pauli_state(x) for x in X]
def kernel(u,v): return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_PL_SVM_pauli:", float(np.mean((yhat-y)**2)))

# SVM + Angle Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_angle' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_angle(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
K_test = K  
yhat = svr.predict(K_test)
print("MSE_SVM_angle:", float(np.mean((yhat-y)**2)))

# SVM + Basis Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_basis' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_basis(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_SVM_basis:", float(np.mean((yhat-y)**2)))

# SVM + Amplitude Embedding (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_amplitude' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_amplitude(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_SVM_amplitude:", float(np.mean((yhat-y)**2)))

# SVM + Fourier Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_fourier' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_fourier(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_SVM_fourier:", float(np.mean((yhat-y)**2)))

# SVM + ZZ Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_zz' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_zz(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_SVM_zz:", float(np.mean((yhat-y)**2)))

# SVM + Pauli Feature Map (Qiskit)

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.svm import SVR

assert 'X' in globals() and 'meta' in globals() and 'sv_pauli' in globals()
N, d = X.shape
path = Path(meta["path"])
if meta.get("target"):
    y = pd.read_csv(path)[meta["target"]].to_numpy(dtype=np.float64)
else:
    y = (X @ np.linspace(0.5,1.5,d)).astype(np.float64)

phis = [sv_pauli(x).data for x in X]
def kernel(u,v): 
    return float(np.abs(np.vdot(u,v))**2)
K = np.array([[kernel(phis[i], phis[j]) for j in range(N)] for i in range(N)])
svr = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
svr.fit(K, y)
yhat = svr.predict(K)
print("MSE_SVM_pauli:", float(np.mean((yhat-y)**2)))